In [1]:
import time
import json
import jsonlines
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


JSON_FILE_PATH = 'C:\\Users\\Stoja\\python_projects\\kp_semantic_search\\ads_json\\'
URL_FILE_PATH = 'C:\\Users\\Stoja\\python_projects\\kp_semantic_search\\ads_json\\urls.txt'


In [2]:
# func for extracting ad name
def extract_ad_name(driver):
    try:
        name = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div/div/div[2]/section[1]/div[2]/section[1]/div[1]/h1').text
        try:
            name.replace('"', 'inc')
            return name or ''
        except:
            return name
    except:
        return 'Nepoznato'
#func for extracting category
def extract_cat(driver):
    try:
        return driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[3]/div/div/div[2]/section[1]/div[1]/div/div[1]/span/div[1]/a[1]').text
    except:
        return 'Nepoznato'

#func for extracting subcategory
def extract_sub_cat(driver):
    try:
        return driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div/div/div[2]/section[1]/div[1]/div/div[1]/span/div[1]/a[2]').text
    except:
        return 'Nepoznato'

#func for extracting id of the ad
def extract_ad_id(driver):
    try:
        return driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div/div/div[2]/section[1]/div[1]/div/div[1]/span/div[1]/span').text
    except:
        return 'Nepoznato'

#func for extracting username
def extract_user(driver):
    try:
        return driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div/div/div[2]/section[1]/div[2]/section[2]/section/div[2]/div[1]/span[2]').text
    except: 
        return 'Nepoznato'

#func for extracting membership
def extract_membership(driver):
    try:
        return driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div/div/div[2]/section[1]/div[2]/section[2]/section/div[2]/div[2]/div/div[1]').text
    except:
        return 'Nepoznato'

#func for extracting place-from
def extract_place(driver):
    try:
        return driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div/div/div[2]/section[1]/div[2]/section[2]/section/div[2]/div[2]/div/div[2]').text
    except:
        return 'Nepoznato'

#func for extracting good reviews
def extract_positive_reviews(driver):
    try:
        return driver.find_element(By.CLASS_NAME, 'ReviewThumbLinks_positive__uoQ7Z').text
    except:
        return 'Nepoznato'


#func for extracting bad reviews
def extract_negative_reviews(driver):
    try:
        return driver.find_element(By.CLASS_NAME, 'ReviewThumbLinks_negative__c9WWb').text
    except:
        return 'Nepoznato'

#func for extracting price
def extract_price(driver):
    try:
        return driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div/div/div[2]/section[1]/div[2]/section[1]/div[3]/div[2]/div[1]/h2').text
    except:
        return 'Nepoznato'

#func for extracting condition
def extract_condition(driver):
    try:
        return driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div/div/div[2]/section[1]/div[2]/section[1]/div[1]/div').text
    except: return 'Nepoznato'
    

#func for extracting images
def extract_images(driver):
    try:
        images = []
        elements = driver.find_elements(By.CLASS_NAME,'GalleryThumbnail_imageGalleryThumbnailInner___ou1n')
        for element in elements:
            image = element.find_element(By.TAG_NAME, 'img').get_attribute("src")
            images.append(image)
        return images
    except:
        return 'Nepoznato'

#func for extracting description
def extract_desc(driver):
    try:
        element =  driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div/div/div[2]/section[1]/div[3]')
        html_content = element.get_attribute('innerHTML')

    # Use BeautifulSoup to parse the HTML content and extract text
        soup = BeautifulSoup(html_content, 'html.parser')
        text_content = soup.get_text(separator='\n')
        return text_content
    except:
        return 'Nepoznato'
    
def extract_url(driver):
    try:
        return driver.current_url
    except:
        return 'Nepoznato'
            

In [4]:
#func for calling all extractor functions

def call(driver):
    ad_name = extract_ad_name(driver)
    ad_id = extract_ad_id(driver)
    condition = extract_condition(driver)
    category = extract_cat(driver)
    sub_cat = extract_sub_cat(driver)
    price = extract_price(driver)
    user = extract_user(driver)
    membership = extract_membership(driver)
    place = extract_place(driver)
    good_reviews = extract_positive_reviews(driver)
    bad_reviews = extract_negative_reviews(driver)
    images = extract_images(driver)
    desc = extract_desc(driver)
    url = extract_url(driver)

    data = {
    'ad': {
        'ad_url': url,
        'ad_name': ad_name,
        'ad_id': ad_id,
        'price': price,
        'condition': condition,
        'category': category,
        'sub_cat': sub_cat,
        'description': desc,
        'images': images
         },
    'user': {
        'name': user,
        'membership': membership,
        'place': place,
        'good_reviews': good_reviews,
        'bad_reviews': bad_reviews
        }
    }
    return data

In [5]:
# def create_json(data):
#     with open(JSON_FILE_PATH, 'w') as json_file:
#         json.dump(data, json_file, indent=4)
#         json_file.write('\n')

def load_urls():
    try:
        with open(URL_FILE_PATH, 'r') as file:
            content = file.read()
            url_list = content.strip().replace('\n', '').split(',')
            return url_list
    except FileNotFoundError:
        print('File not found!')

In [13]:
file1 = 'ads1.jsonl'
file2 = 'ads2.jsonl'
file3 = 'ads3.jsonl'
file4 = 'ads4.jsonl'

url_list = load_urls()
list1 = url_list[:25]
list2 = url_list[25:50]
list3 = url_list[50:75]
list4 = url_list[75:]

In [14]:
print(len(list1))
print(len(list2))
print(len(list3))
print(len(list4))


25
25
25
25


In [16]:
# Setup ChromeDriver
def create_jsonl(list, file_name):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    with jsonlines.open(JSON_FILE_PATH  + file_name, 'w') as json_file:    
        for url in list:
            driver.get(url)

            # Wait for the page to load completely
            time.sleep(5)  # Adjust time as necessary

            # Extract the page source
            data = call(driver)
            json_file.write(data)

    # Close the browser
    driver.quit()

In [20]:
# create_jsonl(list1, file1)
# create_jsonl(list2, file2)
# create_jsonl(list3, file3)
# create_jsonl(list4, file4)